# Brute Force KNN for Binary Classification

Creating a KNN from scratch and testing it on the wine quality dataset

In [1]:
import math

In [8]:
def euclidean_distance(vec1, vec2):
    """
    Calculates the Euclidean distance between two vectors. The Euclidean distance is the "ordinary" straight-line
    distance between two points in Euclidean space. This measure is often used in machine learning to assess
    the similarity between vectors, with a wide range of applications in clustering, classification, and more.

    Parameters:
    - vec1 (list or array-like): The first vector.
    - vec2 (list or array-like): The second vector, which must be of the same length as vec1.

    Returns:
    - float: The Euclidean distance between the two vectors.
    """
    # Ensure the vectors are of the same length
    if len(vec1) != len(vec2):
        raise ValueError("Vectors must be of the same length")

    # Calculate the squared differences and sum them
    squared_diffs = [(v1 - v2) ** 2 for v1, v2 in zip(vec1, vec2)]
    sum_squared_diffs = sum(squared_diffs)

    # Return the square root of the sum of squared differences
    return math.sqrt(sum_squared_diffs)

In [9]:
def manhattan_distance(vec1, vec2):
    """
    Calculates the Manhattan distance between two vectors. The Manhattan distance is defined as the sum
    of the absolute differences between corresponding elements of the vectors. This distance metric is
    often used in geometry, coding theory, and many other fields.

    Parameters:
    - vec1 (list or array-like): The first vector.
    - vec2 (list or array-like): The second vector. It must be of the same length as vec1.

    Returns:
    - int or float: The Manhattan distance between the two vectors.
    """
    # Ensure the vectors are of the same length
    if len(vec1) != len(vec2):
        raise ValueError("Vectors must be of the same length")

    # Calculate the absolute differences and sum them
    abs_diffs = [abs(v1 - v2) for v1, v2 in zip(vec1, vec2)]
    return sum(abs_diffs)

In [10]:
def accuracy_and_error(vec_predictions, vec_actual):
    """
    Calculates the accuracy and generalization error (interpreted as error rate) for a set of predictions
    against the actual labels in a binary classification task. Accuracy is the proportion of correct predictions
    over the total number of predictions, and the error rate is calculated as 1 minus the accuracy.

    Parameters:
    - vec_predictions (list or array-like): Predicted labels by the classification model.
    - vec_actual (list or array-like): Actual labels from the dataset.

    Returns:
    - A tuple containing two floats:
        - accuracy (float): The accuracy of the predictions.
        - error_rate (float): The generalization error, calculated as 1 minus the accuracy.
    """
    # Ensure the vectors are of the same length
    if len(vec_predictions) != len(vec_actual):
        raise ValueError("Vectors must be of the same length")

    # For every prediction that is equal to the actual value, add 1 and find the sum
    correct_predictions = sum(1 for pred, actual in zip(
        vec_predictions, vec_actual) if pred == actual)
    total_predictions = len(vec_predictions)
    # Calculate the proportion of correct predictions
    accuracy = correct_predictions / total_predictions

    # Calculate error rate as 1 - accuracy
    error_rate = 1 - accuracy

    return accuracy, error_rate

In [11]:
def precision(true_positives, false_positives):
    """
    Calculates the precision metric for a binary classification task.
    Precision is defined as the ratio of true positives to the sum of true positives and false positives.

    Parameters:
    - true_positives (int): The count of true positive predictions.
    - false_positives (int): The count of false positive predictions.

    Returns:
    - precision (float): The calculated precision value. Returns 0 if the denominator is 0, preventing division by zero.
    """
    if true_positives + false_positives == 0:
        return 0
    return true_positives / (true_positives + false_positives)


def recall(true_positives, false_negatives):
    """
    Calculates the recall metric for a binary classification task.
    Recall is defined as the ratio of true positives to the sum of true positives and false negatives.

    Parameters:
    - true_positives (int): The count of true positive predictions.
    - false_negatives (int): The count of false negative predictions.

    Returns:
    - recall (float): The calculated recall value. Returns 0 if the denominator is 0, preventing division by zero.
    """
    if true_positives + false_negatives == 0:
        return 0
    return true_positives / (true_positives + false_negatives)


def f1_score(precision, recall):
    """
    Calculates the F1 score for a binary classification task.
    The F1 score is the harmonic mean of precision and recall.

    Parameters:
    - precision (float): The precision value.
    - recall (float): The recall value.

    Returns:
    - F1 Score (float): The calculated F1 score. Returns 0 if the sum of precision and recall is 0, preventing division by zero.
    """
    # Fixing the original formula to correctly calculate the F1 score
    if precision + recall == 0:
        return 0
    return (2 * precision * recall) / (precision + recall)

In [12]:
def confusion_matrix(vec_actual, vec_predictions):
    """
    Computes the confusion matrix for a set of predictions against the actual labels
    for a binary classification task. The confusion matrix is represented as a tuple
    containing counts of true negatives, false positives, false negatives, and true positives.

    Parameters:
    - vec_actual (list): Actual labels from the dataset, where 1 represents
      the positive class and 0 represents the negative class.
    - vec_predictions (list): Predicted labels by the classification model,
      where 1 represents a prediction for the positive class and 0 for the negative class.

    Returns:
    - A tuple of four integers: (true_neg, false_pos, false_neg, true_pos) representing the
      counts of true negatives, false positives, false negatives, and true positives, respectively.
    """

    # cover the case where the denominator is 0
    if len(vec_actual) != len(vec_predictions):
        return ValueError("Vectors must be of the same length")
    
    true_pos, true_neg, false_pos, false_neg = 0, 0, 0, 0
    for actual, pred in zip(vec_actual, vec_predictions):
        if actual == 1 and pred == 1: true_pos += 1
        elif actual == 1 and pred == 0: false_neg += 1
        elif actual == 0 and pred == 1: false_pos += 1
        else: true_neg += 1
        
    return true_neg, false_pos, false_neg, true_pos

In [13]:
def roc_curve(actual, predictions): #TODO might need to convert to numpy arr
    """
    Compute ROC curve data.
    
    Parameters:
    - actual (list): of true binary labels.
    - predictions (list): of scores or probabilities for the positive class.
    
    Returns:
    - fpr: Array of false positive rates.
    - tpr: Array of true positive rates.
    - thresholds: Array of thresholds used.
    """
    thresholds = sorted(predictions)[::-1]
    fpr, tpr = [], []
    
    for threshold in thresholds:
        # new predictions based on the threshold
        preds = [p for p in predictions if p >= threshold ]
        
        # call the confusion matrix function
        tn, fp, fn, tp = confusion_matrix(actual, preds)
        
        current_tpr = tp/(tp+fn) if (tp + fn) > 0 else 0
        current_fpr = fp/(fp+tn) if (fp + tn) > 0 else 0
        
        tpr.append(current_tpr)
        fpr.append(current_fpr)
        
    # return fpr and tpr in an increasing order
    return sorted(fpr), sorted(tpr), thresholds

In [ ]:
def auc_roc(fpr, tpr):
    """
    Calculates the Area Under the Curve (AUC) for the Receiver Operating Characteristic (ROC) curve.
    This function uses the trapezoidal rule to approximate the integral under the ROC curve, which
    represents the AUC. The AUC is a measure of the ability of a classifier to distinguish between
    classes and is used as a summary of the ROC curve.

    Parameters:
    - fpr (list): An array of false positive rates, sorted in increasing order.
    - tpr (list): An array of true positive rates, sorted in increasing order.

    Returns:
    - auc (float): The approximated Area Under the Curve (AUC) value.
    """
    # Ensure the inputs are of the same length
    if len(fpr) != len(tpr):
        raise ValueError("The length of FPR and TPR arrays must be the same")

    # Calculate the AUC using the trapezoidal rule
    auc = 0.0
    for i in range(1, len(fpr)):
        # The area of each trapezoid is calculated and added to the total AUC
        auc += (fpr[i] - fpr[i-1]) * (tpr[i] + tpr[i-1]) / 2

    return auc